# AQI Data Extraction and Processing

In [20]:
import pandas as pd

This is the data we got from https://tspcb.cgg.gov.in/Pages/Envdata.aspx which was provided by NASSCOM

In [21]:
Data = pd.read_csv("newdataset.csv")
Data.head(20)

# Selecting Region ,Month and Year Column 
Data = Data[['Region','Month','Year']]

In [22]:
Data.head()

,Region,Month,Year
0,Adilabad,JAN,2023
1,Adilabad,FEB,2023
2,Nizamabad,JAN,2023
3,Nizamabad,FEB,2023
4,Warangal,JAN,2023


In [23]:
from datetime import datetime

# Generating Starting Date and Ending Date of every month of every year
# using Month and Year column

Data['Start_Date'] = pd.to_datetime(Data['Month'] + ' ' + Data['Year'].astype(str))
Data['End_Date'] = pd.to_datetime(Data['Month'] + ' ' + Data['Year'].astype(str)) + pd.offsets.MonthEnd(1)

# Changing the format of the date 
# yy-mm-dd -->  dd-mm-yy

Data['Start_Date'] = Data['Start_Date'].dt.strftime('%d-%m-%y')
Data['End_Date'] = Data['End_Date'].dt.strftime('%d-%m-%y')



In [24]:
Data.head()

,Region,Month,Year,Start_Date,End_Date
0,Adilabad,JAN,2023,01-01-23,31-01-23
1,Adilabad,FEB,2023,01-02-23,28-02-23
2,Nizamabad,JAN,2023,01-01-23,31-01-23
3,Nizamabad,FEB,2023,01-02-23,28-02-23
4,Warangal,JAN,2023,01-01-23,31-01-23


In [25]:
import datetime

# converting "dd-mm-yy" into UNIX time 
# Append the data to a new Start_Date_Unix column
unix_Startdate = []
for sd in Data['Start_Date']:
    s_dts = datetime.datetime.strptime(sd, "%d-%m-%y")
    unix_Startdate.append(int(s_dts.timestamp()))
Data['Start_Date_Unix'] = unix_Startdate

# converting "dd-mm-yy" into UNIX time
# Append the data to a new End_Date_Unix column
unix_Enddate = []
for ed in Data['End_Date']:
    e_dts = datetime.datetime.strptime(ed, "%d-%m-%y")
    unix_Enddate.append(int(e_dts.timestamp()))
Data['End_Date_Unix'] = unix_Enddate


In [26]:
# Creating a Column with postfix location
Data['Full Location'] = Data.loc[:,('Region')] + ", Telangana, India"
Data.head()

,Region,Month,Year,Start_Date,End_Date,Start_Date_Unix,End_Date_Unix,Full Location
0,Adilabad,JAN,2023,01-01-23,31-01-23,1672511400,1675103400,"Adilabad, Telangana, India"
1,Adilabad,FEB,2023,01-02-23,28-02-23,1675189800,1677522600,"Adilabad, Telangana, India"
2,Nizamabad,JAN,2023,01-01-23,31-01-23,1672511400,1675103400,"Nizamabad, Telangana, India"
3,Nizamabad,FEB,2023,01-02-23,28-02-23,1675189800,1677522600,"Nizamabad, Telangana, India"
4,Warangal,JAN,2023,01-01-23,31-01-23,1672511400,1675103400,"Warangal, Telangana, India"


In [27]:
import geopy
from geopy.geocoders import Nominatim
locator = Nominatim(user_agent="myGeocoder")

# Defining a function to get the latitude and longitude values of a given Full location
def getLatLong(loc):
   
    result = "Not Found"
    location = locator.geocode(loc)
    if(location != None):
        result = "lat={}&lon={}".format(location.latitude, location.longitude)
    return result

# Getting the latitude and longitude values 
# Apply the getLatLong function to the 'Full Location' column in Data, and create a new 'LatLong' column with the results
Data['LatLong'] = Data['Full Location'].apply(lambda x: getLatLong(x))

In [28]:
Data.head()

,Region,Month,Year,Start_Date,End_Date,Start_Date_Unix,End_Date_Unix,Full Location,LatLong
0,Adilabad,JAN,2023,01-01-23,31-01-23,1672511400,1675103400,"Adilabad, Telangana, India",lat=19.5&lon=78.5
1,Adilabad,FEB,2023,01-02-23,28-02-23,1675189800,1677522600,"Adilabad, Telangana, India",lat=19.5&lon=78.5
2,Nizamabad,JAN,2023,01-01-23,31-01-23,1672511400,1675103400,"Nizamabad, Telangana, India",lat=18.75&lon=78.25
3,Nizamabad,FEB,2023,01-02-23,28-02-23,1675189800,1677522600,"Nizamabad, Telangana, India",lat=18.75&lon=78.25
4,Warangal,JAN,2023,01-01-23,31-01-23,1672511400,1675103400,"Warangal, Telangana, India",lat=17.9820644&lon=79.5970954


In [29]:
# saving the data as csv file for future use and can save processing time

Data.to_csv("latlong_updated.csv")

This data is used to extract air quality data based on the Region and the starting and ending points. The following data is to fetch data through Open Weather API.

In [30]:
import urllib.request, urllib.parse, urllib.error
import json

# sample url of OpenWeather api
# http://api.openweathermap.org/data/2.5/air_pollution/history?lat={lat}&lon={lon}&start={start}&end={end}&appid={API key} 

key = "c07a9fcaab2d950fbcc19fef00a77360"
serviceURL = "http://api.openweathermap.org/data/2.5/air_pollution/history?"

In [31]:
for index, row in Data.iterrows():
    # Extract the latitude and longitude from the LatLong column
    LatLong = row['LatLong']
    
    # Extract the start and end date in Unix format
    start_date = row['Start_Date_Unix']
    end_date = row['End_Date_Unix']
    
    # Construct the API URL with the required parameters
    url = f"{serviceURL}{LatLong}&start={start_date}&end={end_date}&appid={key}"
    
    # Send a request to the API URL and receive the response
    response = urllib.request.urlopen(url)
    
    # Parse the response JSON data into a dataframe    
    data = json.loads(response.read().decode())
    df = pd.json_normalize(data, record_path=['list'], meta=['coord'])
    
    # Append the data to a CSV file
    df.to_csv("apidata.csv", mode='a', header=False, index=False)

The data collected through Open Weather API is saved into a .csv file to use it further

In [73]:
# Reading data from the csv file

import pandas as pd
import datetime

Data = pd.read_csv("apidata.csv")
col = ["Date","AQI","CO","NO","NO2","O3","SO2","PM2.5","PM10","NH3","Coord"]
Data.columns = col

In [74]:
Data

,Date,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3,Coord
0,1672516800,5,640.87,0.00,14.91,21.46,1.94,104.19,113.01,14.31,"{'lon': 78.5, 'lat': 19.5}"
1,1672520400,5,620.84,0.01,14.39,16.45,1.73,103.43,112.38,15.45,"{'lon': 78.5, 'lat': 19.5}"
2,1672524000,5,594.14,0.02,13.88,12.16,1.24,97.86,106.59,14.57,"{'lon': 78.5, 'lat': 19.5}"
3,1672527600,5,547.41,0.02,11.65,12.87,1.06,92.22,100.19,10.77,"{'lon': 78.5, 'lat': 19.5}"
4,1672531200,5,520.71,0.02,9.94,16.45,1.37,90.76,97.94,7.16,"{'lon': 78.5, 'lat': 19.5}"
...,...,...,...,...,...,...,...,...,...,...,...
132336,1609336800,5,614.17,0.00,9.51,57.94,4.17,166.48,174.98,5.13,"{'lon': 80.3333, 'lat': 17.5}"
132337,1609340400,5,634.19,0.00,9.17,52.21,3.70,169.40,178.33,5.64,"{'lon': 80.3333, 'lat': 17.5}"
132338,1609344000,5,620.84,0.00,8.23,50.07,3.49,163.43,172.44,5.83,"{'lon': 80.3333, 'lat': 17.5}"
132339,1609347600,5,587.46,0.00,7.28,55.79,4.05,148.19,157.18,5.45,"{'lon': 80.3333, 'lat': 17.5}"


In [75]:
# converting the UNIX Date into "dd-mm-yy" format

formatted_date = []
for un in Data['Date']:
    dt = datetime.datetime.fromtimestamp(un)
    formatted_date.append(dt.strftime('%d-%m-%y'))
Data['Date'] = formatted_date    

In [76]:
Data["Coord"].unique()

array(["{'lon': 78.5, 'lat': 19.5}", "{'lon': 78.25, 'lat': 18.75}",
       "{'lon': 79.5971, 'lat': 17.9821}",
       "{'lon': 79.1328, 'lat': 18.4348}",
       "{'lon': 80.3333, 'lat': 17.5}"], dtype=object)

In [77]:
#The lattitude and longitude is mapped with the respective location

dic = {"{'lon': 78.5, 'lat': 19.5}":"Adilabad, Telangana, India", 
       "{'lon': 78.25, 'lat': 18.75}":"Nizamabad, Telangana, India",
       "{'lon': 79.5971, 'lat': 17.9821}":"Warangal, Telangana, India",
       "{'lon': 79.1328, 'lat': 18.4348}":"Karimnagar, Telangana, India",
       "{'lon': 80.3333, 'lat': 17.5}":"Khammam, Telangana, India"}

In [78]:
#The Cordinates are replaced with their respective location

Data["Coord"] = Data["Coord"].map(dic)

In [79]:
Data = Data.rename(columns={'Coord': 'Location'})

In [80]:
Data['Date'] = pd.to_datetime(Data['Date'])

In [81]:
Data.to_csv('alldata.csv',index = False)

In [42]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132341 entries, 0 to 132340
Data columns (total 11 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   Date      132341 non-null  datetime64[ns]
 1   AQI       132341 non-null  int64         
 2   CO        132341 non-null  float64       
 3   NO        132341 non-null  float64       
 4   NO2       132341 non-null  float64       
 5   O3        132341 non-null  float64       
 6   SO2       132341 non-null  float64       
 7   PM2.5     132341 non-null  float64       
 8   PM10      132341 non-null  float64       
 9   NH3       132341 non-null  float64       
 10  Location  132341 non-null  object        
dtypes: datetime64[ns](1), float64(8), int64(1), object(1)
memory usage: 11.1+ MB


In [43]:
Data.head()

,Date,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3,Location
0,2023-01-01,5,640.87,0.00,14.91,21.46,1.94,104.19,113.01,14.31,"Adilabad, Telangana, India"
1,2023-01-01,5,620.84,0.01,14.39,16.45,1.73,103.43,112.38,15.45,"Adilabad, Telangana, India"
2,2023-01-01,5,594.14,0.02,13.88,12.16,1.24,97.86,106.59,14.57,"Adilabad, Telangana, India"
3,2023-01-01,5,547.41,0.02,11.65,12.87,1.06,92.22,100.19,10.77,"Adilabad, Telangana, India"
4,2023-01-01,5,520.71,0.02,9.94,16.45,1.37,90.76,97.94,7.16,"Adilabad, Telangana, India"


In [44]:
# Group the data by date and location, and calculate the mean values for each group to remove duplicate values

grouped_data = Data.groupby(['Date', 'Location']).mean()

In [51]:
# Reset the index of the grouped data to create a new dataframe with one row per date for each location

new_data = grouped_data.reset_index(drop=False)

In [52]:
new_data.head()

,Date,Location,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,2020-01-12,"Adilabad, Telangana, India",4.958333,456.175417,0.458333,11.532500,73.984583,14.119167,64.316667,71.244583,7.498333
1,2020-01-12,"Karimnagar, Telangana, India",5.000000,720.283333,1.470000,17.677500,48.816667,7.001667,101.994167,115.704583,14.710833
2,2020-01-12,"Khammam, Telangana, India",5.000000,463.965000,0.108750,6.395833,56.237083,3.274583,75.047500,81.250833,3.192083
3,2020-01-12,"Nizamabad, Telangana, India",4.875000,584.543333,0.411250,9.455417,59.755000,10.259167,74.220000,87.468750,15.049583
4,2020-01-12,"Warangal, Telangana, India",5.000000,641.704167,0.205417,15.177917,52.822500,6.458333,88.811250,98.957917,5.284583


In [53]:
# Convert the floating-point data in AQI to integers using ceiling and floor

for column in new_data.columns:
    if new_data['AQI'].dtype == 'float64':
        new_data['AQI'] = new_data['AQI'].apply(lambda x: int(x) if x == int(x) else int(x) + 1 if x - int(x) >= 0.5 else int(x))

In [55]:
import math

#Creating a function to calculate AQI 
def calculate_aqi(so2_conc, O3_conc, no2_conc, pm10_conc, pm25_conc, nh3_conc):
    
    # Calculate AQI for each pollutant
    aqi_so2 = calculate_aqi_pollutant(so2_conc, "SO2")
    aqi_O3 = calculate_aqi_pollutant(O3_conc, "O3")
    aqi_no2 = calculate_aqi_pollutant(no2_conc, "NO2")
    aqi_pm10 = calculate_aqi_pollutant(pm10_conc, "PM10")
    aqi_pm25 = calculate_aqi_pollutant(pm25_conc, "PM2.5")
    aqi_nh3 = calculate_aqi_pollutant(nh3_conc, "NH3")

    # Determine the overall AQI by taking the maximum AQI value
    aqi = max(aqi_so2, aqi_O3, aqi_no2, aqi_pm10, aqi_pm25)
    
    # aqi = statistics.mea(aqi)
    return aqi


def calculate_aqi_pollutant(conc, pollutant):
    
    # Calculate AQI for each pollutant
    
    # Define the AQI breakpoints and corresponding values for each pollutant
    
    if pollutant == "SO2":
        breakpoints = [0, 40, 80, 380, 800, 1600]
        aqi_values = 	[0, 50, 100, 200, 300, 400, 500]
    elif pollutant == "O3":
        breakpoints = 	[0, 50, 100, 168, 208, 748]
        aqi_values = [0, 50, 100, 150, 200, 300, 400]
    elif pollutant == "NO2":
        breakpoints = [0, 40, 80, 180, 280, 400]
        aqi_values = [0, 50, 100, 200, 300, 400, 500]
    elif pollutant == "PM10":
        breakpoints = [0, 50, 100, 250, 350, 430]
        aqi_values = [0, 50, 100, 150, 200, 300, 400]
    elif pollutant == "PM2.5":
        breakpoints = [0, 40, 80, 120, 250, 450]
        aqi_values = [0, 50, 100, 150, 200, 300,400]
    elif pollutant == "NH3":
        breakpoints = [0, 200, 400, 800, 1200, 1800]
        aqi_values = [0, 50, 100, 200, 300, 400, 500]
        
    else:
        raise ValueError("Invalid pollutant type")
       
    # Calculate the AQI for the given concentration value
    
    if conc <= breakpoints[0]:
        aqi = 0
    elif conc > breakpoints[-1]:
        aqi = 500
    else:
        for i in range(len(breakpoints)-1):
            if conc > breakpoints[i] and conc <= breakpoints[i+1]:
                aqi = (aqi_values[i+1] - aqi_values[i]) / (breakpoints[i+1] -  breakpoints[i]) * (conc - breakpoints[i]) + aqi_values[i]

    return math.ceil(aqi) if (aqi - math.floor(aqi)) >= 0.5 else math.floor(aqi)
    
    

    #Indian standard breakpoints to calculate AQI
    
    # if pollutant == "SO2":
    #     breakpoints = 	[0, 20,40, 80, 160,400,600]
    #     aqi_values = 	[0, 50, 100 , 150, 200, 300, 500]
    # elif pollutant == "O3":
    #     breakpoints = 		[0, 20,50,100,200,400,600]
    #     aqi_values = [0, 50, 100 , 150, 200, 300, 500]
    # elif pollutant == "NO2":
    #     breakpoints = [0, 20,40, 80, 160,400,600]
    #     aqi_values = [0, 50, 100 , 150, 200, 300, 500]
    # elif pollutant == "PM10":
    #     breakpoints = [0,30,60,100,200,300,400 ]
    #     aqi_values = [0, 50, 100 , 150, 200, 300, 500]
    # elif pollutant == "PM2.5":
    #     breakpoints = [0, 20,40,60,100,200,300]
    #     aqi_values = [0, 50, 100 , 150, 200, 300, 500]
    # elif pollutant == "NH3":
    #     breakpoints = [0, 200, 400, 800, 1200, 1800]
    #     aqi_values = [0, 50, 100 , 150, 200, 300, 500]  
        
        
    #US standard breakpoints to calculate AQI   
        
    # if pollutant == "SO2":
    #     breakpoints = 	[0, 40, 80, 380, 800, 1600]
    #     aqi_values = 	[0, 50, 100, 150, 200, 300, 400]
    # elif pollutant == "O3":
    #     breakpoints = 	[0, 54, 70, 85, 105, 200, 405]
    #     aqi_values = [0, 50, 100, 150, 200, 300, 400]
    # elif pollutant == "NO2":
    #     breakpoints = 	[0, 40, 80, 180, 280, 400]
    #     aqi_values = [0, 50, 100, 150, 200, 300, 400]
    # elif pollutant == "PM10":
    #     breakpoints = 	[0, 54, 154, 254, 354, 424]
    #     aqi_values = [0, 50, 100, 150, 200, 300, 400]
    # elif pollutant == "PM2.5":
    #     breakpoints = [0, 12, 35.4, 55.4, 150.4, 250.4, 350.4, 500.4]
    #     aqi_values = [0, 50, 100, 150, 200, 300, 400]
    # elif pollutant == "NH3":
    #     breakpoints = [0, 200, 400, 800, 1200, 1800]
    #     aqi_values = [0, 50, 100, 150, 200, 300, 400] 
    

In [56]:
# Applying the function to the dataset

new_data['AQI'] = new_data.apply(lambda row: calculate_aqi(row['SO2'], row['O3'], row['NO2'], row['PM10'], row['PM2.5'], row['NH3']), axis=1)

In [57]:
new_data.head()

,Date,Location,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
0,2020-01-12,"Adilabad, Telangana, India",80,456.175417,0.458333,11.532500,73.984583,14.119167,64.316667,71.244583,7.498333
1,2020-01-12,"Karimnagar, Telangana, India",127,720.283333,1.470000,17.677500,48.816667,7.001667,101.994167,115.704583,14.710833
2,2020-01-12,"Khammam, Telangana, India",94,463.965000,0.108750,6.395833,56.237083,3.274583,75.047500,81.250833,3.192083
3,2020-01-12,"Nizamabad, Telangana, India",93,584.543333,0.411250,9.455417,59.755000,10.259167,74.220000,87.468750,15.049583
4,2020-01-12,"Warangal, Telangana, India",111,641.704167,0.205417,15.177917,52.822500,6.458333,88.811250,98.957917,5.284583


In [58]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3975 entries, 0 to 3974
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      3975 non-null   datetime64[ns]
 1   Location  3975 non-null   object        
 2   AQI       3975 non-null   int64         
 3   CO        3975 non-null   float64       
 4   NO        3975 non-null   float64       
 5   NO2       3975 non-null   float64       
 6   O3        3975 non-null   float64       
 7   SO2       3975 non-null   float64       
 8   PM2.5     3975 non-null   float64       
 9   PM10      3975 non-null   float64       
 10  NH3       3975 non-null   float64       
dtypes: datetime64[ns](1), float64(8), int64(1), object(1)
memory usage: 341.7+ KB


In [59]:
new_data.tail(10)

,Date,Location,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
3965,2023-12-01,"Adilabad, Telangana, India",102,626.126667,0.492917,21.173750,74.357917,12.417500,81.341250,88.908750,26.025417
3966,2023-12-01,"Karimnagar, Telangana, India",148,884.812083,0.817500,34.516667,67.191667,16.960417,118.477917,136.091250,21.917083
3967,2023-12-01,"Khammam, Telangana, India",141,588.575417,0.187917,9.422083,78.953333,13.868333,112.933333,124.292083,8.152917
3968,2023-12-01,"Nizamabad, Telangana, India",104,716.945000,0.579583,13.287083,54.347083,6.513750,82.950417,97.210000,21.221250
3969,2023-12-01,"Warangal, Telangana, India",130,715.414167,0.970417,17.427083,55.064167,8.205417,104.374583,116.298333,11.734583
3970,2023-12-02,"Adilabad, Telangana, India",80,402.907500,0.357083,8.285417,79.795000,9.086667,47.749583,66.591250,10.138750
3971,2023-12-02,"Karimnagar, Telangana, India",91,495.672917,0.481667,12.202500,90.719583,17.990417,58.332917,77.517917,18.829167
3972,2023-12-02,"Khammam, Telangana, India",83,343.382083,0.104167,2.982917,75.802083,7.885417,66.299583,76.054167,4.109583
3973,2023-12-02,"Nizamabad, Telangana, India",79,440.597500,0.144167,6.539167,79.468333,8.400417,49.185833,62.796250,12.590000
3974,2023-12-02,"Warangal, Telangana, India",82,441.432083,0.171667,6.049167,73.270000,7.676250,65.351667,80.888333,8.086667


In [71]:
import datetime 

# Sorting the data till today

df = new_data[new_data["Date"]<= datetime.datetime.today()]

In [72]:
df.tail()

,Date,Location,AQI,CO,NO,NO2,O3,SO2,PM2.5,PM10,NH3
3880,2023-03-02,"Adilabad, Telangana, India",70,565.211250,0.622500,20.360000,70.088750,12.405000,50.934583,60.712917,11.169583
3881,2023-03-02,"Karimnagar, Telangana, India",86,601.371250,0.988750,24.018750,64.700833,28.093333,68.824583,80.454167,9.978333
3882,2023-03-02,"Khammam, Telangana, India",66,418.901250,0.099583,6.044583,52.749167,3.569167,52.509167,58.062917,6.887083
3883,2023-03-02,"Nizamabad, Telangana, India",73,455.617917,0.379583,9.297500,72.918333,8.105833,39.035833,49.436250,13.032500
3884,2023-03-02,"Warangal, Telangana, India",61,524.460000,0.255000,10.634583,54.271250,6.532500,48.564167,56.716667,9.408750


In [82]:
df.to_csv("AQI.csv", index=False)

In [83]:
df = pd.read_csv('AQI.csv')

In [85]:
new_names = {'Adilabad, Telangana, India': 'Adilabad', 'Karimnagar, Telangana, India': 'Karimnagar', 'Khammam, Telangana, India': 'Khammam',
            'Nizamabad, Telangana, India': 'Nizamabad','Warangal, Telangana, India':'Warangal'}

In [86]:
df['Location'].replace(new_names, inplace=True)

In [88]:
df.to_csv("AQI.csv", index=False)